### build_staging_from_merged.py -> 표준화/정제(region 통일, 카테고리 분리, 이름 공백행 제거 등)

In [1]:
# -*- coding: utf-8 -*-
"""
staging 빌더 스크립트
- 입력: ./_merged_csv 폴더 내 각 원본(DB) 통합 CSV들(예: 포털DB_merged.csv 등)
- 처리: 소스별 스키마를 표준 스키마로 정규화 + 카테고리 분리/JSON화 + region 표준화
- 출력: staging_all.utf8.csv (UTF-8) / staging_all.csv (CP949)
"""
import os
import re
import json
import pandas as pd
from pathlib import Path
from typing import Dict, List, Optional

# =========================================
# 경로 설정 (현재 작업 디렉터리 기준 상대경로)
# =========================================
MERGED_DIR = Path("./_merged_csv")         # *_merged.csv 들이 있는 폴더
OUT_PATH_UTF8  = MERGED_DIR / "staging_all.utf8.csv"
OUT_PATH_CP949 = MERGED_DIR / "staging_all.csv"

# =========================================
# 지역 정규화 사전
#  - 다양한 표기(서울특별시/서울시/서울 등)를 단일 키(예: "서울")로 통일
# =========================================
REGION_VARIANTS: Dict[str, List[str]] = {
    "서울": ["서울특별시", "서울시", "서울"],
    "부산": ["부산광역시", "부산시", "부산"],
    "대구": ["대구광역시", "대구시", "대구"],
    "인천": ["인천광역시", "인천시", "인천"],
    "광주": ["광주광역시", "광주시", "광주"],
    "대전": ["대전광역시", "대전시", "대전"],
    "울산": ["울산광역시", "울산시", "울산"],
    "세종": ["세종특별자치시", "세종시", "세종"],
    "경기": ["경기도", "경기"],
    "강원": ["강원특별자치도", "강원도", "강원"],
    "충북": ["충청북도", "충북"],
    "충남": ["충청남도", "충남"],
    "전북": ["전라북도", "전북"],
    "전남": ["전라남도", "전남"],
    "경북": ["경상북도", "경북"],
    "경남": ["경상남도", "경남"],
    "제주": ["제주특별자치도", "제주도", "제주"],
}

# =========================================
# 유틸
# =========================================
def read_csv_auto(path: Path) -> Optional[pd.DataFrame]:
    """
    CSV 인코딩 자동 판별 로더.
    - utf-8-sig 먼저 시도 → 실패 시 cp949 재시도.
    - 둘 다 실패하면 None 반환.
    """
    if not path.exists():
        return None
    for enc in ("utf-8-sig", "cp949"):
        try:
            return pd.read_csv(path, dtype=str, encoding=enc)
        except Exception:
            continue
    return None

# 카테고리 분리: ",", "/", "·", "."
CAT_SPLIT_RE = re.compile(r"\s*(?:,|/|·|\.)\s*")

def split_category(s: Optional[str]) -> List[str]:
    """
    원본 업종/분류 문자열을 구분자(, / · .)로 분리하고
    공백 제거 + 중복 제거(순서 보존)하여 리스트로 반환.
    """
    if not isinstance(s, str):
        return []
    raw = [t.strip() for t in CAT_SPLIT_RE.split(s) if t and t.strip()]
    seen, out = set(), []
    for t in raw:
        if t not in seen:
            seen.add(t)
            out.append(t)
    return out

def to_json_list(lst: List[str]) -> str:
    """파이썬 리스트 → JSON 문자열('["a","b"]')로 직렬화."""
    return json.dumps(lst, ensure_ascii=False)

def normalize_region(region: Optional[str], address: Optional[str]) -> str:
    """
    지역 문자열을 REGION_VARIANTS 기준으로 표준 키로 통일.
    - region 값이 비었으면 address에서 추론.
    - 매칭 실패 시 빈 문자열("") 반환.
    """
    cand = (region or "").strip()
    addr = (address or "").strip()

    def _match(text: str) -> Optional[str]:
        if not text:
            return None
        for key, vars_ in REGION_VARIANTS.items():
            for v in vars_:
                if v in text:
                    return key
        return None

    reg = _match(cand) or _match(addr)
    return reg or ""

def drop_blank_name(df: pd.DataFrame, name_col: str) -> pd.DataFrame:
    """
    이름 컬럼이 공백/결측인 행 제거.
    - 소스별 name_col은 SOURCES[name_col]에서 지정.
    """
    mask = df[name_col].astype(str).str.strip() != ""
    return df.loc[mask].copy()

# =========================================
# 소스별 표준화 규칙
#  - 각 소스의 파일명과, 이름/주소/날짜/업종(분류) 컬럼명을 정의
#  - ref_date는 date_cols 우선순위대로 첫 값 채택
# =========================================
SOURCES = {
    "포털DB": {
        "file": "포털DB_merged.csv",
        "name_col": "상호명",
        "date_cols": ["업데이트날짜", "업데이트"],
        "addr_col": "주소",
        "raw_cat_col": "업종",
    },
    "기업DB": {
        "file": "기업DB_merged.csv",
        "name_col": "업체명",
        "date_cols": ["등록일"],
        "addr_col": "주소",
        "raw_cat_col": "업종",
    },
    "인허가DB": {
        "file": "인허가DB_merged.csv",
        "name_col": "업체명",
        "date_cols": ["업데이트날짜", "업데이트"],
        "addr_col": "주소",
        "raw_cat_col": "업종",
    },
    "법인DB": {
        "file": "법인DB_merged.csv",
        "name_col": "업체명",
        "date_cols": ["업데이트날짜", "업데이트"],
        "addr_col": "주소",
        "raw_cat_col": "업종",
    },
    "마켓DB": {
        "file": "마켓DB_merged.csv",
        "name_col": "업체명",
        "date_cols": ["등록일"],
        "addr_col": "주소",
        "raw_cat_col": "분류",
    },
}

# 스테이징 표준 스키마(컬럼 순서 고정)
STD_COLS = [
    "region", "source_db", "merchant_code",
    "name", "address", "ref_date", "raw_category", "category"
]

# =========================================
# 메인 변환
# =========================================
def build_staging() -> pd.DataFrame:
    """
    *_merged.csv 들을 읽어 표준 스키마로 변환 후 하나의 DataFrame으로 병합.
    처리 내용:
      - region 정규화(없으면 주소에서 추론)
      - category 분리(구분자 기반) → JSON 문자열로 저장
      - 이름 공백/결측 행 제거
      - ref_date는 소스별 후보 중 첫 값 채택
    """
    frames = []
    for source_db, cfg in SOURCES.items():
        path = MERGED_DIR / cfg["file"]
        df = read_csv_auto(path)
        if df is None:
            print(f"[INFO] skip: 파일 없음 → {path.name}")
            continue

        # 결측은 일단 공백으로 채움(후속 가공 일관성)
        df = df.fillna("")

        # 소스별 컬럼명 매핑
        name_col  = cfg["name_col"]
        addr_col  = cfg["addr_col"]
        raw_col   = cfg["raw_cat_col"]
        date_cols = cfg["date_cols"]

        # 표준 스키마 그리기
        out = pd.DataFrame()

        # region: 존재하면 사용, 없으면 주소에서 추론(특히 기업DB 대비)
        has_region = "region" in df.columns
        out["region"] = [
            normalize_region(
                df.loc[i, "region"] if has_region else "",
                df.loc[i, addr_col] if addr_col in df.columns else ""
            )
            for i in df.index
        ]

        # source_db: 고정 값
        out["source_db"] = source_db

        # merchant_code: 원본 통합 단계에서 이미 부여된 코드 사용(없으면 공백)
        out["merchant_code"] = df["merchant_code"] if "merchant_code" in df.columns else ""

        # name / address
        out["name"] = df[name_col] if name_col in df.columns else ""
        out["address"] = df[addr_col] if addr_col in df.columns else ""

        # ref_date: date_cols 우선순위대로 첫 값 채움
        ref_date_series = pd.Series([""] * len(df))
        for dc in date_cols:
            if dc in df.columns:
                empty_mask = ref_date_series.eq("")
                ref_date_series.loc[empty_mask] = df[dc].astype(str).fillna("")
        out["ref_date"] = ref_date_series

        # 원본 업종/분류 → raw_category
        out["raw_category"] = df[raw_col] if raw_col in df.columns else ""

        # category: 구분자 분리 리스트를 JSON 문자열로 직렬화
        out["category"] = out["raw_category"].map(lambda s: to_json_list(split_category(s)))

        # 이름 공백/결측 행 제거
        out = drop_blank_name(out, "name")

        # 표준 컬럼 순서 재배치 + 결측 공백
        out = out.reindex(columns=STD_COLS).fillna("")
        frames.append(out)

        print(f"[OK] {source_db}: {len(out):,}행")

    # 소스에서 단 한 건도 못 읽었을 때
    if not frames:
        print("[WARN] 수집된 데이터가 없습니다.")
        return pd.DataFrame(columns=STD_COLS)

    # 최종 병합 + 결측 공백
    staging = pd.concat(frames, axis=0, ignore_index=True, sort=False).fillna("")
    return staging

# =========================================
# 실행 진입점
# =========================================
if __name__ == "__main__":
    # 출력 폴더 보장
    MERGED_DIR.mkdir(parents=True, exist_ok=True)

    # 스테이징 DF 생성
    staging = build_staging()

    # 저장 (UTF-8 + CP949)
    staging.to_csv(OUT_PATH_UTF8, index=False, encoding="utf-8-sig")

    # CP949 호환본(손실 문자는 무시하고 저장)
    staging_cp = staging.applymap(
        lambda x: x.encode("cp949", errors="ignore").decode("cp949") if isinstance(x, str)
        else ("" if pd.isna(x) else x)
    )
    staging_cp.to_csv(OUT_PATH_CP949, index=False, encoding="cp949")

    print(f"\n✅ 저장 완료 → {OUT_PATH_UTF8.name} / {OUT_PATH_CP949.name} (총 {len(staging):,}행)")


[OK] 포털DB: 4,321,425행
[OK] 기업DB: 46,841행
[OK] 인허가DB: 7,826,824행
[OK] 법인DB: 730,602행
[OK] 마켓DB: 546,150행


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_19168\2176795729.py:262: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  staging_cp = staging.applymap(



✅ 저장 완료 → staging_all.utf8.csv / staging_all.csv (총 13,471,842행)
